Download data from ERA5 reanalysis dataset

[API guide](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels-monthly-means?tab=form)

Notes:
- I usually run this in Colab to save files to Google Drive easily
- You can use a website like [bboxfinder](#http://bboxfinder.com/#) to get the coordinates of the desired area, but you need to pay attention to the ERA 5 API correct order: [North, West, South, East ]

In [ ]:
!pip install cdsapi --quiet

!pip install cdstoolbox-remote

# to be able to read grib files
!pip install cfgrib --quiet
!pip install eccodes --quiet
!sudo apt-get install libeccodes0  --quiet

In [ ]:
from google.colab import drive
import os
import cdsapi
import cfgrib
import xarray as xr

from time import sleep

mount = '/content/drive'
drive.mount('/content/drive')

#Switch to the directory on the Google Drive that you want to use
drive_root = mount  +"/MyDrive/Colab Notebooks/Climate"
%cd $drive_root

In [ ]:
API_KEY =  ""
cds = cdsapi.Client(url="https://cds.climate.copernicus.eu/api/v2",key=API_KEY)

In [ ]:
area_download = [2.46, -71.97, -33.26,-36.64] #brazil area

variables = [
    '2m_temperature',
    'total_precipitation',
    'mean_sea_level_pressure'
]

folder_name = '3_variables'

years_to_download = [str(i) for i in range(1983,2024)]

times_available = [ '00:00', '01:00', '02:00',
                  '03:00', '04:00', '05:00',
                  '06:00', '07:00', '08:00',
                  '09:00', '10:00', '11:00',
                  '12:00', '13:00', '14:00',
                  '15:00', '16:00', '17:00',
                  '18:00', '19:00', '20:00',
                  '21:00', '22:00', '23:00',]

times_to_get = times_available[::3]

#download blocks
block_size = 6
block_start, block_end = 0, block_size

In [1]:
datasets = []
while block_start < len(years_to_download):

  cur_years = years_to_download[block_start:block_end ]

  start_y, end_y = cur_years[0],cur_years[-1]

  print('Downloading',cur_years)

  dataset_name =f"era_{start_y}_{end_y}"
  dataset_name =f'data/brazil/{folder_name}/{dataset_name}.grib'

  datasets.append(dataset_name)
  
  data = cds.retrieve(
      'reanalysis-era5-single-levels',
      {
          'product_type': 'reanalysis',
          'variable': variables,

          'year': cur_years,

          'month': [
              '01', '02', '03','04', '05', '06',
              '07', '08', '09','10', '11', '12'],

          'day': [
              '01', '02', '03','04', '05', '06',
              '07', '08', '09','10', '11', '12',
              '13', '14', '15','16', '17', '18',
              '19', '20', '21','22', '23', '24',
              '25', '26', '27','28', '29', '30','31'],

          'time': times_to_get,

          'area': [2.46, -71.97, -33.26,-36.64], #Brazil box http://bboxfinder.com/#-33.268014,-71.975849,2.469466,-36.648545

          'format': 'grib',

      },dataset_name)

  block_start = block_end
  block_end += block_size

  print('-- finished --')

  sleep(30)